In [1]:
from modules.transformer import Transformer
from options.base_options import BaseOptions
from models.Transform2D import Transform2D
from datasets.shape_net import ShapeNet
import torch
from blocks.patch_encoder import PatchEncoder
from einops import rearrange
from tqdm.notebook import tqdm
from models.Transform2D import Transform2D
%load_ext autoreload
%autoreload 2

In [2]:
dataset = ShapeNet(cat="airplane")
len(dataset)
train_ds = dataset
model = Transform2D()
train_dataloader = torch.utils.data.DataLoader(
        train_ds,   # Datasets return data one sample at a time; Dataloaders use them and aggregate samples into batches
        batch_size=16,   # The size of batches is defined here
        shuffle=True,    # Shuffling the order of samples is useful during training to prevent that the network learns to depend on the order of the input data
        num_workers=4,   # Data is usually loaded in parallel by num_workers
        pin_memory=True,  # This is an implementation detail to speed up data uploading to the GPU
        # worker_init_fn=train_dataset.worker_init_fn  TODO: Uncomment this line if you are using shapenet_zip on Google Colab
    )

In [3]:

for batch_idx, batch in tqdm(enumerate(train_dataloader)):
        #import pdb;pdb.set_trace()
        model(batch)

0it [00:00, ?it/s]

> c:\users\youssef\repos\tum\ss23\3dml\exercises\project\src\models\transform2d.py(48)forward()
     46 
     47         #import pdb;pdb.set_trace()
---> 48         self.x = x
     49         return x
     50 

torch.Size([2, 121, 768])


In [ ]:
from utils.visualizations import visualize_images

#import pdb;pdb.set_trace()
#images = rearrange(images, 'bs c d w -> bs d w c')
visualize_images(y)

y

In [ ]:
images = rearrange(images, 'bs c d w -> bs d w c')
visualize_images(images)

In [ ]:
y.shape
images.shape
images[0][:,:,0]
images.dtype
y.dtype
# y = y.long()


In [ ]:
import torch.nn.functional as F
x = torch.randn(1, 4, 137, 137)
kc, kh, kw = 4, 16, 16  # kernel size
dc, dh, dw = 1, 1, 1  # stride
# Pad to multiples of 32
x = F.pad(x, (x.size(2)%kw // 2, x.size(2)%kw // 2,
              x.size(1)%kh // 2, x.size(1)%kh // 2,
              x.size(0)%kc // 2, x.size(0)%kc // 2))

patches = x.unfold(1, kc, dc).unfold(2, kh, dh).unfold(3, kw, dw)
unfold_shape = patches.size()
patches = patches.contiguous().view(-1, kc, kh, kw)
print(patches.shape)

# Reshape back
patches_orig = patches.view(unfold_shape)
output_c = unfold_shape[1] * unfold_shape[4]
output_h = unfold_shape[2] * unfold_shape[5]
output_w = unfold_shape[3] * unfold_shape[6]
patches_orig = patches_orig.permute(0, 1, 4, 2, 5, 3, 6).contiguous()
patches_orig = patches_orig.view(1, output_c, output_h, output_w)

patches_orig.shape

In [ ]:

from utils.model_utils import summarize_model
total_params = sum(p.numel() for p in model.transformer.parameters())
total_params

In [ ]:

x = torch.ones((3,5,5,5,2)) 

In [ ]:
from einops import rearrange
x[0][0][0][0]
y = rearrange(x, 'bs c1 c2 c3 c4 -> bs c4 c1 c2 c3 ')
y.shape

In [ ]:
y
sfy = torch.nn.Softmax(dim=1)
sfx = torch.nn.Softmax(dim=-1)
y2 = sfy(y)
x2 = sfx(x)

In [ ]:
y2
x2
y3 = rearrange(y2,'bs c4 c1 c2 c3 ->bs c1 c2 c3 c4')

In [ ]:
y3==x2


In [ ]:
n_deconvfilter = [128, 128, 128, 64, 32, 2]
filter_shape = (n_deconvfilter[1], 3, 3, 3)
filter_shape = (n_deconvfilter[3], 3, 3, 3)
padding = [0, int((filter_shape[1] - 1) / 2), 0, int((filter_shape[2] - 1) / 2),
                             int((filter_shape[3] - 1) / 2) ] 

In [ ]:
padding

In [ ]:
from functools import reduce
from operator import __add__
kernel_sizes = (16, 16)
conv_padding = reduce(__add__, 
    [(k // 2 + (k - 2 * (k // 2)) - 1, k // 2) for k in kernel_sizes[::-1]])

In [ ]:
conv_padding

In [ ]:
from einops import rearrange,repeat
import torch
embedd = torch.tensor([[1,2,3],[4,5,6]])
embedd.shape
pos = torch.tensor([[10,10,10]])
pos.shape
positions =  repeat(pos,'n d -> (repeat n) d', repeat=2)

In [ ]:
positions.shape

In [ ]:
positions

In [ ]:
embedd + positions